In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

from siuba import *
import pandas as pd
import geopandas as gpd
import datetime as dt

from rt_analysis import rt_parser
from rt_analysis import rt_filter_map_plot

import shared_utils

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
from rt_analysis import sccp_tools
import numpy as np

In [3]:
from tqdm.notebook import tqdm

# About the Metrics

The schedule-based metric is a daily average of the sum of median trip stop delays along the corridor. To further explain, we take each corridor trip that we have data for and look at the delay in comparison to the schedule at each stop, after subtracting off any delay present as the trip entered the corridor. For each trip we then take the median delay of all stops along the corridor, and sum these medians to create the metric.

The speed-based metric is a daily average of the sum of delays for each trip traversing the corridor as compared to a reference speed of 16 miles per houar. To further explain, we take each corridor trip that we have data for and calculate the hypothetical time it would take for that trip to traverse the corridor at a speed of 16 mph. The difference between the actual time it took for the trip to traverse the corridor and that hypothetical time is the speed-based delay for that trip, and we sum those delays to create the metric. This metric is intended to provide a more consistent basis for comparison independent of scheduling practices.

In other words, if we expect a hypothetical bus lane/signal priority/payment system etc to increase corridor speeds to 16mph, this is how much time we could save per day.

With corridor attached, generate both metrics using `RtFilterMapper.corridor_metrics()`

## Common Date Range (2022-23) and convienience function

In [4]:
date_range = np.arange('2022-04-30', '2022-05-09', dtype='datetime64[D]')

In [5]:
type(date_range)

numpy.ndarray

In [6]:
def generate_export_data(itp_id, date_range, pbar):
    for date in date_range:
        analysis_date = date.astype(dt.date)
        check_ran = shared_utils.rt_utils.get_operators(analysis_date, [itp_id])
        if check_ran[itp_id] == 'already_ran':
            continue
        rt_day = rt_parser.OperatorDayAnalysis(itp_id, analysis_date, pbar)
        # don't worry about the CRS not set UserWarning
        rt_day.export_views_gcs()

# AC / Lake

In [7]:
# itp_id = 4 ## AC Transit
# itp_id = 159 ## Lake Transit

In [8]:
# corridor = gpd.read_file('./corridors_sccp_lpp/CMCP_StudyAreaBoundary.geojson') ## alameda

# Samtrans

In [9]:
# itp_id = 290 # Samtrans

In [10]:
# corridor = gpd.read_file('./corridors_sccp_lpp/peninsula.geojson') ## foster city / hayward park

# Metro (Inglewood)

In [11]:
# itp_id = 182
# corridor = gpd.read_file('./corridors_sccp_lpp/inglewood_map.geojson')

# SacRT/SJRTD, post-refactor

In [12]:
pbar = tqdm()

0it [00:00, ?it/s]

In [13]:
sac_rt_itp_id = 273
sjrtd_itp_id = 284

In [14]:
sac_date_range = np.arange('2022-02-05', '2022-02-14', dtype='datetime64[D]')

In [15]:
%%capture --no-stdout

generate_export_data(sac_rt_itp_id, sac_date_range, pbar)

already ran: 273
already ran: 273
already ran: 273
already ran: 273
not yet run: 273


KeyboardInterrupt: 

In [16]:
%%capture --no-stdout

generate_export_data(sjrtd_itp_id, date_range, pbar)

already ran: 284
already ran: 284
already ran: 284
already ran: 284
already ran: 284
already ran: 284
already ran: 284
already ran: 284
already ran: 284


In [17]:
corridor = gpd.read_file('./corridors_sccp_lpp/sacrt_sjrtd_nov_2.geojson')

In [18]:
corridor.explore()

In [19]:
sac_corridor_metrics = sccp_tools.sccp_average_metrics(itp_id=273, date_range=sac_date_range, corridor=corridor)

failed for date: 2022-02-05
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/273_2022-02-05.parquet
failed for date: 2022-02-06
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/273_2022-02-06.parquet
failed for date: 2022-02-07
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/273_2022-02-07.parquet
failed for date: 2022-02-08
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/273_2022-02-08.parquet
failed for date: 2022-02-09
calitp-analytics-data/data-analyses/rt_delay/rt_trips/273_2022-02-09.parquet
failed for date: 2022-02-10
calitp-analytics-data/data-analyses/rt_delay/rt_trips/273_2022-02-10.parquet
failed for date: 2022-02-11
calitp-analytics-data/data-analyses/rt_delay/rt_trips/273_2022-02-11.parquet
failed for date: 2022-02-12
calitp-analytics-data/data-analyses/rt_delay/rt_trips/273_2022-02-12.parquet
failed for date: 2022-02-13
calitp-analytics-data/data-analyses/rt_delay/rt_trips/273_2022-02-13.parquet


/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars


In [20]:
sjrtd_corridor_metrics = sccp_tools.sccp_average_metrics(itp_id=284, date_range=date_range, corridor=corridor)

failed for date: 2022-04-30
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/284_2022-04-30.parquet
failed for date: 2022-05-01
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/284_2022-05-01.parquet
failed for date: 2022-05-02
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/284_2022-05-02.parquet
failed for date: 2022-05-03
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/284_2022-05-03.parquet
failed for date: 2022-05-04
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/284_2022-05-04.parquet
failed for date: 2022-05-05
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/284_2022-05-05.parquet
failed for date: 2022-05-06
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/284_2022-05-06.parquet
failed for date: 2022-05-07
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/284_2022-05-07.parquet
failed for date: 2022-05-08
calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/284_20

/opt/conda/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
/opt/conda/lib/python3.10/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars


In [21]:
date_range

array(['2022-04-30', '2022-05-01', '2022-05-02', '2022-05-03',
       '2022-05-04', '2022-05-05', '2022-05-06', '2022-05-07',
       '2022-05-08'], dtype='datetime64[D]')

In [22]:
rt_filter_map_plot.from_gcs?

Signature: rt_filter_map_plot.from_gcs(itp_id, analysis_date, pbar=None)
Docstring:
Generates RtFilterMapper from cached artifacts in GCS. Generate using rt_analysis.OperatorDayAnalysis.export_views_gcs()
    
File:      ~/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py
Type:      function


In [23]:
rt_day = rt_filter_map_plot.from_gcs(273, dt.date(2022, 2, 6))

FileNotFoundError: calitp-analytics-data/data-analyses/rt_delay/stop_delay_views/273_2022-02-06.parquet

In [35]:
rt_day.add_corridor(corridor)

In [37]:
# rt_day.quick_map_corridor()

In [27]:
rt_day.corridor_metrics()

{'schedule_metric_minutes': 2.4166666666666665,
 'speed_metric_minutes': -3.5781744005258327}